In [1]:
!pip install dataclasses

from dataclasses import dataclass
import typing
import sympy
import sympy.physics.units as units
sympy.init_printing()

In [10]:
class Symbols:
    @dataclass
    class Item:
        name: str
        description: str
        value: sympy.Expr
        units: sympy.Expr
        dimension: sympy.physics.units.Dimension
    
    def __init__(self, *symbols):
        self._symbols = {}
        for symbol in symbols:
            if isinstance(symbol, Symbols):
                self._symbols.update(symbol._symbols)
            elif isinstance(symbol, Symbols.Item):
                self._add(symbol)
            else:
                raise ValueError(symbol)
        
    def _add(self, **kwargs):
        item =  Symbols.Item(**kwargs)
        self._symbols[item.name] = item

    def __getattr__(self, name):
        return self._symbols[name].value
        
    def __str__(self):
        return '\n'.join(
            f" - {item.name}: {item.description} ({item.dimension.name} - {str(item.units)})"
            for item in self._symbols.values()
        )
    def __repr__(self):
        return f'Symbols({", ".join(str(symbol) for symbol in self._symbols.values())})'
    
    @staticmethod
    def _units(expr):
        return sympy.Expr(expr).subs({x: 1 for x in expr.args if not x.has(sympy.physics.units.Quantity)})
    
    @staticmethod
    def _dimension(expr):
        return sympy.physics.units.Quantity._collect_factor_and_dimension(expr)[1]

class Variables(Symbols):
    """
    Auxiliary symbols that have no fixed value. e.g., time, position, etc 
    """
    def __init__(self, **parameters):
        super().__init__()
        for name, entry in parameters.items():
            units, description = entry
            symbol = sympy.Symbol(name)
            
            self._add(
                name = name,
                description = description,
                value = symbol,
                units = units,
                dimension = Symbols._dimension(units)
            )

class Parameters(Symbols):
    """
    Symbols we are trying to calculate from experimental data
    
    These are the main outputs from the algorithm
    """
    def __init__(self, **parameters):
        super().__init__()
        for name, entry in parameters.items():
            units, description = entry
            symbol = sympy.Symbol(name)
            
            self._add(
                name = name,
                description = description,
                value = symbol,
                units = units,
                dimension = Symbols._dimension(units)
            )

class Functions(Symbols):
    """
    Functions whose values depend on Parameters and are der
    """
    def __init__(self, **parameters):
        super().__init__()
        for name, entry in parameters.items():
            expr, parameters, description = entry
            symbol = sympy.Function(name)(*parameters)
            
            self._add(
                name = name,
                description = description,
                value = symbol,
                units = units,
                dimension = Symbols._dimension(units)
            )
            
class ExperimentalData(Symbols):
    """
    Symbols we have experimental values for
    """
    def __init__(self, **parameters):
        super().__init__()
        for name, entry in parameters.items():
            units, description = entry
            symbol = sympy.Symbol(name)
            
            self._add(
                name = name,
                description = description,
                value = symbol,
                units = units,
                dimension = Symbols._dimension(units)
            )


In [ ]:
symbols = Symbols(
    Variables(
        t       = (units.second, "Time"),
        r       = (units.centimeter, "Radial position in the solid phase"),
        
        
        q       = (units.milligram/units.milligram, "concentration of adsorbate in the solid phase at time t and position r"),
    ),
    Parameters(
        Bi      = (1, "Biot number"),
        C_eq    = (units.milligram/units.liter, "Equilibrium concentration of adsorbate in the liquid phase"),
        C       = (units.milligram/units.liter, "Concentration of adsorbate in the liquid phase"),  # Varia em Tempo e raio
        C_b     = (units.milligram/units.liter, "Mean concentration of adsorbate in the liquid phase"),  # Varia em Tempo ?
        C_s     = (units.milligram/units.liter, "Concentration of adsorbate in the particle surface"),  # Varia em Tempo
        D_eff   = (units.centimeter**2/units.second,    "Effective diffusivity"),
        D_pore  = (units.millimeter, "Average pore diameter"),
        d_z     = (units.milligram/units.milliliter, "Material density"),
        k_1     = (units.liter/units.milligram/units.minute, "Intrinsic rate constant"),
        k_2     = (1/units.second, "Intrinsic rate constant"),
        
        k_D     = (units.milligram/units.liter, "Dissociation constant"),
        k_f     = (units.centimeter/units.second, "Film coefficient"),
        m_Z     = (units.milligram, "Mass of adsorbent"),
        q_eq    = (units.milligram/units.milligram, "Equilibrium concentration of adsorbate in the solid phase"),
        q_m     = (units.milligram/units.milligram, "Maximum adsorption capacity of the adsorvent"),
        
        R_sl    = (units.percent, "Solid:Liquid ratio in the reactor"),
        rq      = (1/units.second, "Adsortion rate"),
        SF      = (units.percent, "Sensitivity factor"),
    ),
    ExperimentalData(
        E_p     = (1, "Particle porosity"),
        C_0     = (units.milligram/units.liter, "Initial concentration of adsorbate in the liquid phase"),
        C_exp   = (units.milligram/units.liter, "Experimental mean concentration of adsorbate in the liquid phase"),  # Varia em Tempo
        R       = (units.centimeter, "Radius of particle"),
        v_obs   = (1/units.second, "Observed reation rate"),  # Varia em Tempo
        V_micro = (units.milliliter/units.milligram, "Volume of micropores"),
        V_pore  = (units.milliliter/units.milligram, "Total volume of pores"),
        V_l     = (units.milliliter, "Liquid volume"),
        V_s     = (units.milliliter, "Solid volume"),
        S_BET   = (units.centimeter**2/units.milligram, "Area BET"),
    ),
    #DifferentialEquations(
    #    eq3_1   = sympy.Eq()
    #)
)

q = sympy.Function("q")(symbols.t)
C_1 = sympy.Function("C_1")(symbols.t)
C_2 = sympy.Function("C_2")(symbols.t)
C_3 = sympy.Function("C_3")(symbols.t)
eqs = (
    sympy.Eq(q.diff(symbols.t), q - C_1),
    sympy.Eq(C_1.diff(symbols.t), q - C_2),
    sympy.Eq(C_2.diff(symbols.t), C_1 - C_3),
    sympy.Eq(C_3.diff(symbols.t), C_2 - C_3),
)

#sympy.solveset(eqs, q, domain=sympy.S.Reals)
sympy.dsolve(eqs, q)


#type(sympy.Function("foo")(rq, symbols.t))
#sympy.Function("foo")(symbols.R)

In [89]:
t = sympy.Symbol("t")
_x = sympy.Function("x")
_y = sympy.Function("y")
_z = sympy.Function("z")
x = _x(t)
y = _y(t)
z = _z(t)
eq = (
    sympy.Eq(x.diff(t), x*y*sympy.sin(t)), 
    sympy.Eq(y.diff(t), y**2*sympy.sin(t)),
    sympy.Eq(z.diff(t), z**2*sympy.sin(t)),
    #sympy.Eq(z.diff(t), x*t)
)
sympy.dsolve(eq)
#{Eq(x(t), -exp(C1)/(C2*exp(C1) - cos(t))), Eq(y(t), -1/(C1 - cos(t)))}

KeyError: F3_

In [ ]:
tf.contrib.integrate.odeint

In [1]:
from data_fit import DataFit
import sympy.physics.units as units

datafit = DataFit()
datafit.add_experimental_data("x", units.meter, "Displacement")
datafit.add_experimental_data("F_exp", units.newton, "Measured Force")
datafit.add_parameter("k", units.newton/units.meter, "Sprint constant")
datafit.add_expr("F", units.newton, datafit.k * datafit.x, "Predicted force")


Expr(newton)


AssertionError: Incompatible dimensions, expected Dimension(force, F), but actually is Dimension(1)